In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,1714
2,Huelva,Confirmados PDIA 14 días,507
3,Huelva,Tasa PDIA 14 días,"99,26714609891863"
4,Huelva,Confirmados PDIA 7 días,241
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,3
629,Municipio de Huelva sin especificar,Total Confirmados,26
630,Municipio de Huelva sin especificar,Curados,1


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1714.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  411.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2711 personas en los últimos 7 días 

Un positivo PCR cada 1579 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,1714.0,241.0,507.0,510743.0,47.186158,99.267146,44.0
Condado-Campiña,612.0,126.0,251.0,155057.0,81.260440,161.875955,22.0
Huelva-Costa,852.0,97.0,200.0,288115.0,33.667112,69.416726,20.0
Lepe,131.0,28.0,56.0,27431.0,102.074296,204.148591,9.0
Huelva (capital),411.0,53.0,91.0,143663.0,36.891893,63.342684,9.0
San Juan del Puerto,64.0,43.0,43.0,9300.0,462.365591,462.365591,6.0
Palma del Condado (La),86.0,17.0,50.0,10761.0,157.977883,464.640833,4.0
Almonte,58.0,4.0,9.0,24191.0,16.535075,37.203919,2.0
Villarrasa,13.0,7.0,9.0,2176.0,321.691176,413.602941,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Manzanilla,19.0,3.0,15.0,2135.0,140.515222,702.576112,0.0
Almendro (El),5.0,1.0,4.0,826.0,121.065375,484.261501,0.0
Palma del Condado (La),86.0,17.0,50.0,10761.0,157.977883,464.640833,4.0
San Juan del Puerto,64.0,43.0,43.0,9300.0,462.365591,462.365591,6.0
Valdelarco,2.0,1.0,1.0,239.0,418.410042,418.410042,0.0
Villarrasa,13.0,7.0,9.0,2176.0,321.691176,413.602941,2.0
Bollullos Par del Condado,106.0,22.0,55.0,14272.0,154.147982,385.369955,1.0
Villanueva de los Castillejos,12.0,5.0,10.0,2820.0,177.304965,354.609929,1.0
Lucena del Puerto,13.0,2.0,8.0,3371.0,59.329576,237.318303,1.0
